# Hyperparameters

Consider the test function of two inputs $f(x_1, x_2) = e^{- (x_1/10)^2 - x_2^2}$. We can see that this function varies more quickly in the $x_1$ direction than the $x_2$ direction, especially if we look at the plot:

In [ ]:
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt

x1 = x2 = np.linspace(-10, 10, 256)
X1, X2 = np.meshgrid(x1, x2)

F = -np.exp(-((X1 / 8) ** 2)) * np.exp(-(X2**2))

plt.pcolormesh(x1, x2, F, shading="auto")
plt.colorbar()
plt.xlabel("x1")
plt.ylabel("x2")

The optimization goes faster if our model understands how the function changes as we change the inputs in different ways. The way it picks up on this is by starting from a general model that could describe a lot of functions, and making it specific to this one by choosing the right hyperparameters. Our Bayesian agent is very good at this, and only needs a few samples to figure out what the function looks like:

In [ ]:
%run -i ../../../examples/prepare_bluesky.py # prepare the bluesky environment

import bloptools
from bloptools.tasks import Task

dofs = bloptools.devices.dummy_dofs(n=2)
bounds = [(-8, 8), (-8, 8)]

task = Task(key="loss", kind="min")


def digestion(db, uid):
    products = db[uid].table()

    for index, entry in products.iterrows():
        products.loc[index, "loss"] = -np.exp(-1e-2 * entry.x1**2) * np.exp(-1e-1 * entry.x2**2)

    return products


agent = bloptools.bayesian.Agent(
    active_dofs=dofs,
    passive_dofs=[],
    active_dof_bounds=bounds,
    tasks=[task],
    digestion=digestion,
    db=db,
)

RE(agent.initialize(acqf="qr", n_init=16))

agent.plot_tasks()

In [ ]:
agent.tasks[0].regressor.covar_module.latent_dimensions

In addition to modeling the fitness of the task, the agent models the probability that an input will be feasible:

In [ ]:
agent.plot_acquisition(strategy=["ei", "pi", "ucb"])

In [ ]:
RE(agent.learn("ei", n_iter=4))
agent.plot_tasks()